In [1]:
import pickle
import numpy as np
from ribbn_scripts.ref_functions.spec_functions import read_network_analyzer_file, get_theta, get_amplitude, s2z



In [2]:
def dbm_to_mV(dbm,Z=50):
    return 1000*np.sqrt(Z/1000)*(10**(dbm/10))

# def dbm_to_V(dbm,ro=50):
#     # return np.sqrt((ro/1000)*(10**(dbm/10)))
#     return watts_to_V(dbm_to_W(dbm), ro=ro)

# def watts_to_V(pw, ro=50):
#     return np.sqrt(pw*ro)

# def V_to_watts(V,rho=50):
#     return V**2/(rho)

# def mV_to_dbm(mV, Z=50):
#     return 20 * np.log10(mV / (1000 * np.sqrt(Z / 1000)))

# def dbm_to_W(dbm): # check once, this formula might only be true for db to W
#     return 10**(dbm/10)  * 1e-3 #Watts


In [3]:
# ch_list = ['1', '3', '4', '6', '7','8']



# if rx=="1":
#     tx_for_vna='v32-3'
#     rx_for_pv='v32-5'
# else: # todo make this as an argument of the function
#     tx_for_vna='v32-5'
#     rx_for_pv='v32-3'

# phases=[]
# amps = []
# attns = []
# for ch in ch_list:

#     tx_dat = read_network_analyzer_file(
#                                 f'{calibration_path}/VNA_Dec2025/'+str(tx_for_vna)+'_channel_b\'' + f"ch_{str(ch)}" + '\'_vna_pwr_15.csv')
#     sl_tx = tx_dat[1] * np.exp(1j * tx_dat[2])
#     gamma = (s2z(sl_tx) - s2z(np.conj(50))) / (s2z(sl_tx) + s2z(50))
#     gamma=1-gamma

#     freq_a = tx_dat[0]
#     p = np.polyfit(freq_a, np.unwrap(np.angle(gamma)), 1)
#     phases.append(np.polyval(p, freq))

#     p = np.polyfit(freq_a, abs(gamma), 1)
#     attns.append(np.polyval(p, freq))
    
#     rx_pv=pickle.load(open(f"{calibration_path}/PV_data_Dec2025/{rx_for_pv}_pv_polynomials_rx.pkl","rb"))
#     dbm_corrected=np.polyval(rx_pv[freq/1e6]["polynomial"],np.log(adc_out))
#     mV_corrected=dbm_to_mV(dbm_corrected)
#     amps.append(mV_corrected)

# th = get_theta(amps, attns, phases) 


In [4]:
with open("processedDF.pkl",'rb') as f:
    processedDF = pickle.load(f)
processedDF

,Rx,Tx,Voltages (mV),Phase1,Phase3,Phase4,Phase6,Phase7,Phase8,Frequency (MHz),Run Exp Num,NumMPPs,Dist (m)
0,Tag1,Tag2,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",194.900,199.820,210.495,184.900,187.340,192.530,845,0,1,1.185
1,Tag1,Tag2,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",195.085,199.435,210.690,185.210,187.800,192.265,845,0,1,1.185
2,Tag1,Tag2,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",195.090,200.045,210.540,184.900,187.535,192.265,845,0,1,1.185
3,Tag1,Tag2,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",195.120,199.705,210.610,185.170,187.305,192.760,845,0,1,1.185
4,Tag1,Tag2,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",195.165,200.770,211.415,185.400,188.070,193.140,845,0,1,1.185
...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,Tag2,Tag1,"[43.98, 19.19, 16.37, 18.88, 16.21, 19.87, 17....",18.880,23.120,15.945,17.660,21.670,25.065,945,7,1,1.185
876,Tag2,Tag1,"[43.98, 19.19, 16.37, 18.88, 16.21, 19.87, 17....",16.480,20.025,13.655,16.865,20.410,25.105,945,7,1,1.185
877,Tag2,Tag1,"[43.98, 19.19, 16.37, 18.88, 16.21, 19.87, 17....",16.515,20.485,14.230,16.100,21.290,25.520,945,7,1,1.185
878,Tag2,Tag1,"[43.98, 19.19, 16.37, 18.88, 16.21, 19.87, 17....",18.155,21.135,13.690,16.555,20.945,25.295,945,7,1,1.185


11

In [29]:
# In a set of 10 MPPs first 5 are from tag1 to tag2, next 5 are from tag2 to tag1. 
# These 5 mpps can be cross multiplied to get 25 phase measurements.

#Dims: (no of experiments (locations), number of unique frequencies, two dims (phase and amp), 25 values (5mpps X 5mpp))
FINAL_DATA=np.zeros([np.max(processedDF['Run Exp Num'])+1, len(np.unique(processedDF["Frequency (MHz)"])), 2, 25])

calibration_path="/Users/manavjeet/git/T2TExperiments/coba/calibrations"
ch_list = ['1', '3', '4', '6', '7','8']
freq_idx=0


for i in range(0,len(processedDF),10):
    if freq_idx>=FINAL_DATA.shape[1]:
        freq_idx=0

    DFoI_1 = processedDF[i:i+5]
    DFoI_2 = processedDF[i+5:i+10]
    
    #Check if the frequency used in same across MPPs
    assert(DFoI_1.iloc[0]['Frequency (MHz)']==DFoI_2.iloc[-1]['Frequency (MHz)'])
    
    #Check if the exp no used in same across MPPs
    assert(DFoI_1.iloc[0]['Run Exp Num']==DFoI_2.iloc[-1]['Run Exp Num'])
    
    freq=DFoI_1.iloc[0]['Frequency (MHz)']*1e6
    exp_no=DFoI_1.iloc[0]['Run Exp Num']
    # if freq!=915e6:
    #     continue
    all_channel_atten=[]
    all_channel_phase=[]
    for row1 in range(5):
        for row2 in range(5):
            # print(ff)
            # ff+=1
            t1_row = DFoI_1.iloc[row1]
            t2_row = DFoI_2.iloc[row2]
            # print(t2_row)
            
            thetas=[]
            amplitudes=[]
            voltages=[]
            for rx in ['1','2']:
                if rx=="1":
                    tx_for_vna='v32-3'
                    rx_for_pv='v32-5'
                    # rx_for_vna='tag4'
                    # tx_for_vna='tag5'
                    # rx_for_pv='tag4'
                else: # todo make this as an argument of the function
                    tx_for_vna='v32-5'
                    rx_for_pv='v32-3'
                    # rx_for_vna='tag5'
                    # tx_for_vna='tag4'
                    # rx_for_pv='tag1'

                phases=[]
                amps = []
                attns = []
                for ch in ch_list:
                    if rx=='1':
                        adc_out=t1_row[f"Phase{ch}"]
                    else:
                        adc_out=t2_row[f"Phase{ch}"]
                    
                    tx_dat = read_network_analyzer_file(
                            f'{calibration_path}/VNA_Dec2025/'+str(tx_for_vna)+'_channel_b\'' + f"ch_{str(ch)}" + '\'_vna_pwr_15.csv')
                    
                    # tx_dat = read_network_analyzer_file(
                    #         f'{calibration_path}/VNA_Oct2024/'+str(tx_for_vna)+'_channel_b\'' + f"ch_{str(ch)}" + '\'_vna_pwr_15.csv')
                    sl_tx = tx_dat[1] * np.exp(1j * tx_dat[2])
                    gamma = (s2z(sl_tx) - s2z(np.conj(50))) / (s2z(sl_tx) + s2z(50))
                    gamma=1-gamma

                    freq_all = tx_dat[0]
                    p = np.polyfit(freq_all, np.unwrap(np.angle(gamma)), 1)
                    phases.append(np.polyval(p, freq))

                    p = np.polyfit(freq_all, abs(gamma), 1)
                    attns.append(np.polyval(p, freq))
                    
                    rx_pv=pickle.load(open(f"{calibration_path}/PV_data_Dec2025/{rx_for_pv}_pv_polynomials_rx.pkl","rb"))
                    # rx_pv=pickle.load(open(f"{calibration_path}/PV_data_Aug2024/{rx_for_pv}_pv_polynomials_rx.pkl","rb"))
                    dbm_corrected=np.polyval(rx_pv[freq/1e6]["polynomial"],np.log(adc_out))
                    mV_corrected=dbm_to_mV(dbm_corrected)
                    amps.append(mV_corrected)
                    
                voltages.append(np.mean(amps))
                
                th = get_theta(amps, attns, phases) 
                thetas.append(th)
                t2tamp = get_amplitude(amps, attns, phases) 
                amplitudes.append(t2tamp)
            
            channel_phase=((thetas[0]+thetas[1])/2) % np.pi
            channel_atten=(amplitudes[0]/voltages[1] + amplitudes[1]/voltages[0])/2
            
            all_channel_phase.append(channel_phase)
            all_channel_atten.append(channel_atten)
            
            # print(f"Channel phase: {channel_phase}")
            # print(f"Channel atten: {channel_atten}")
            # print(f"Freq = {freq}")
            # print(thetas)
            # print(amplitudes)
            # print(voltages)
            # print()            
            # break
    #     break
    # break
    all_channel_atten=np.array(all_channel_atten)
    all_channel_phase=np.array(all_channel_phase)
    
    FINAL_DATA[exp_no, freq_idx, 0,:]=all_channel_atten
    FINAL_DATA[exp_no, freq_idx, 1,:]=all_channel_phase
    
    # print(len(all_channel_atten))
    
    freq_idx+=1

In [28]:
FINAL_DATA

array([[[[ 0.03310323,  0.03609461,  0.03116365, ...,  0.03127008,
           0.03333725,  0.03303577],
         [ 1.38867332,  1.42568777,  1.39186391, ...,  1.38693145,
           1.41986238,  1.3903725 ]],

        [[ 0.06764343,  0.05314798,  0.06122057, ...,  0.06152032,
           0.05844086,  0.05897767],
         [-0.91902197, -0.83010311, -0.89289479, ..., -0.85294878,
          -0.88631954, -0.85253884]],

        [[ 0.01400333,  0.04320843,  0.06309958, ...,  0.06412499,
           0.05386452,  0.05425586],
         [ 2.54177055,  2.56736215,  2.71574401, ...,  2.72840193,
           2.61293965,  2.71226024]],

        ...,

        [[ 0.03536492,  0.03643008,  0.0382355 , ...,  0.03859634,
           0.03586308,  0.03608958],
         [ 1.51882018,  1.5366762 ,  1.52448048, ...,  1.54667593,
           1.53959997,  1.56504493]],

        [[ 0.1243571 ,  0.12285757,  0.1372649 , ...,  0.13743606,
           0.12111421,  0.13792067],
         [-1.3542873 , -1.32562624, -1.329

In [16]:
d=1.185
theta=((2*np.pi*d*845e6)/3e8)%np.pi
print(theta)

2.1221458374999074
